In [ ]:
%load_ext autoreload
%autoreload 2

In [101]:
import os
os.environ['KERAS_BACKEND'] = "torch"
import keras
import keras.backend as K
import keras.layers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from modules.dataset import DatasetFactory
from modules.encoding import NLFEncoder

In [ ]:
dataset_dir = "../data/SAbDab"

df_match = pd.read_csv(os.path.join(dataset_dir, "data.csv"), sep=";")
df_seq = pd.read_csv(os.path.join(dataset_dir, "sequences.csv"), sep=";")

In [ ]:
df_match.head()

### Filter 

Remove keys which does not have both an antigen and an anticorps.

In [ ]:
df_filtered = df_match[df_match["ab"].isin(df_seq["seq_id"]) & df_match["ag"].isin(df_seq["seq_id"])]
df_filtered.shape

In [ ]:
df_match.shape

## Encoding

### NLF

We want to encode the sequence first, and retrieve the encoded for each sequence in the join later.

In [ ]:
encoder = NLFEncoder()

In [ ]:
seq = df_seq["sequence"].values
encoded_seq = encoder.encode(seq)

In [ ]:
seq_id_to_nlf = {seq_id: nlf for seq_id, nlf in zip(df_seq["seq_id"], encoded_seq)}

In [ ]:

x_ag = df_filtered["ag"].map(lambda x: seq_id_to_nlf[x])
x_ab = df_filtered["ab"].map(lambda x: seq_id_to_nlf[x])

## Siamese network

In [ ]:
# Convolutional modules
filters = 96

conv01 = layers.Conv1D(filters, 11, padding='same', activation="relu")
mp1 = layers.MaxPooling1D(3)
conv02 = layers.Conv1D(filters*2, 7, padding='same', activation="relu")
mp2 = layers.MaxPooling1D(3)
conv03 = layers.Conv1D(filters*4, 3, padding='same', activation="relu")
mp3 = layers.MaxPooling1D(3)
conv04 = layers.Conv1D(filters*2, 3, padding='same', activation="relu")
mp4 = layers.MaxPooling1D(3)

gru = layers.Bidirectional(layers.CuDNNGRU(filters, return_sequences=False))
if not gpus:
    gru = layers.Bidirectional(layers.GRU(filters, return_sequences=False))

def siamese_propagation(x):
    #Enter your code here to create a Siamese module
    
    return x